In [175]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style # for styling the graphs
#_______________________________________________________________________________________________________________
# style.available (to know the available list of styles)
style.use('ggplot') # chosen style
plt.rc('xtick',labelsize=13) # to globally set the tick size
plt.rc('ytick',labelsize=13) # to globally set the tick size
# To print multiple outputs together
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Change column display number during print
pd.set_option('display.max_columns', 500)
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
# To display float with 2 decimal, avoid scientific printing
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
import sweetviz
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report
from sklearn.metrics import roc_auc_score ,mean_squared_error,accuracy_score,classification_report,roc_curve,confusion_matrix,precision_score,f1_score
import itertools
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [176]:
path='./Train.csv'
path1='./Test.csv'

In [177]:
train=pd.read_csv(path)
train

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch,MatchWinner
0,5,4,37,4,Home,Away,Second,First,Dec,4
1,1,14,84,7,Neutral,Neutral,First,Second,Sep,1
2,9,15,47,9,Home,Away,First,Second,Feb,9
3,7,2,102,6,Home,Away,First,Second,Aug,2
4,6,8,46,5,Home,Away,First,Second,Aug,6
...,...,...,...,...,...,...,...,...,...,...
2503,5,15,89,3,Neutral,Neutral,Second,First,Jun,5
2504,1,14,138,0,Home,Away,First,Second,Jan,1
2505,14,5,135,15,Home,Away,Second,First,Apr,14
2506,15,14,64,16,Home,Away,First,Second,Nov,14


In [178]:
test=pd.read_csv(path1)
test

,Team1,Team2,Stadium,HostCountry,Team1_Venue,Team2_Venue,Team1_Innings,Team2_Innings,MonthOfMatch
0,2,4,34,1,Home,Away,First,Second,Oct
1,14,1,19,15,Home,Away,First,Second,Mar
2,9,10,130,14,Neutral,Neutral,Second,First,Dec
3,9,10,8,9,Home,Away,First,Second,Dec
4,5,15,130,14,Neutral,Neutral,First,Second,Oct
...,...,...,...,...,...,...,...,...,...
1070,15,5,64,16,Home,Away,First,Second,Jun
1071,1,12,95,0,Home,Away,First,Second,Feb
1072,5,10,43,1,Neutral,Neutral,First,Second,Mar
1073,10,13,111,0,Neutral,Neutral,Second,First,Mar


In [179]:
X=train.drop('MatchWinner',1)
y=train['MatchWinner']
Xt=test
train.columns

Index(['Team1', 'Team2', 'Stadium', 'HostCountry', 'Team1_Venue',
       'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch',
       'MatchWinner'],
      dtype='object')

In [180]:
def frequency_encoding(column, df, df_test=None):
    frequencies = df[column].value_counts().reset_index()
    df_values = df[[column]].merge(frequencies, how='left', 
                                   left_on=column, right_on='index').iloc[:,-1].values
    if df_test is not None:
        df_test_values = df_test[[column]].merge(frequencies, how='left', 
                                                 left_on=column, right_on='index').fillna(1).iloc[:,-1].values
    else:
        df_test_values = None
    return df_values, df_test_values

for column in ['Team1', 'Team2', 'Stadium', 'HostCountry', 'Team1_Venue',
       'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch']:
    train_values, test_values = frequency_encoding(column, X, Xt)
    X[column+'_counts'] = train_values
    Xt[column+'_counts'] = test_values

In [181]:
X.drop(['Team1', 'Team2', 'Stadium', 'HostCountry', 'Team1_Venue',
       'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch'],1,inplace=True)

In [182]:
Xt.drop(['Team1', 'Team2', 'Stadium', 'HostCountry', 'Team1_Venue',
       'Team2_Venue', 'Team1_Innings', 'Team2_Innings', 'MonthOfMatch'],1,inplace=True)

In [183]:
new_train=pd.concat([X,y],1)
new_train

,Team1_counts,Team2_counts,Stadium_counts,HostCountry_counts,Team1_Venue_counts,Team2_Venue_counts,Team1_Innings_counts,Team2_Innings_counts,MonthOfMatch_counts,MatchWinner
0,364,193,13,275,1797,1797,1255,1255,208,4
1,386,335,5,5,710,710,1253,1253,147,1
2,253,195,21,220,1797,1797,1253,1253,307,9
3,50,87,46,50,1797,1797,1253,1253,133,2
4,49,25,23,37,1797,1797,1253,1253,133,6
...,...,...,...,...,...,...,...,...,...,...
2503,364,195,3,267,710,710,1255,1255,176,5
2504,386,335,108,407,1797,1797,1253,1253,380,1
2505,171,238,15,196,1797,1797,1255,1255,197,14
2506,127,335,87,135,1797,1797,1253,1253,176,14


In [210]:
new_test=Xt

new_train=new_train[['Team1_counts','Team2_counts','MatchWinner']]
new_test=new_test[['Team1_counts','Team2_counts']]

In [211]:
from sklearn.preprocessing import PolynomialFeatures
X=new_train.drop('MatchWinner',1)
y=new_train['MatchWinner']
poly = PolynomialFeatures(3)

In [212]:
new_train_poly=poly.fit_transform(X)

In [213]:
new_train_poly=pd.DataFrame(new_train_poly)

In [214]:
new_train_poly=pd.concat([new_train_poly,y],1)

In [215]:
new_train_poly

,0,1,2,3,4,5,6,7,8,9,MatchWinner
0,1.00,364.00,193.00,132496.00,70252.00,37249.00,48228544.00,25571728.00,13558636.00,7189057.00,4
1,1.00,386.00,335.00,148996.00,129310.00,112225.00,57512456.00,49913660.00,43318850.00,37595375.00,1
2,1.00,253.00,195.00,64009.00,49335.00,38025.00,16194277.00,12481755.00,9620325.00,7414875.00,9
3,1.00,50.00,87.00,2500.00,4350.00,7569.00,125000.00,217500.00,378450.00,658503.00,2
4,1.00,49.00,25.00,2401.00,1225.00,625.00,117649.00,60025.00,30625.00,15625.00,6
...,...,...,...,...,...,...,...,...,...,...,...
2503,1.00,364.00,195.00,132496.00,70980.00,38025.00,48228544.00,25836720.00,13841100.00,7414875.00,5
2504,1.00,386.00,335.00,148996.00,129310.00,112225.00,57512456.00,49913660.00,43318850.00,37595375.00,1
2505,1.00,171.00,238.00,29241.00,40698.00,56644.00,5000211.00,6959358.00,9686124.00,13481272.00,14
2506,1.00,127.00,335.00,16129.00,42545.00,112225.00,2048383.00,5403215.00,14252575.00,37595375.00,14


In [190]:
from tqdm.notebook import tqdm 
from sklearn.metrics import log_loss

X=new_train_poly.drop('MatchWinner',1)
y=new_train_poly['MatchWinner']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=9)

lr=LogisticRegression(random_state=9)
dt=DecisionTreeClassifier(random_state=9)
rf=RandomForestClassifier(random_state=9)
lg=LGBMClassifier(random_state=9)
xgb=XGBClassifier(random_state=9)
catb=CatBoostClassifier(random_state=9)

models=[lr,dt,rf,lg,xgb,catb]


for model in models:
    model.fit(X_train,y_train)
    y_pred=model.predict_proba(X_test)
    LOG_LOSS=log_loss(y_test,model.predict_proba(X_test))
    print('LOG LOSS:',LOG_LOSS)
    cv=cross_val_score(model,X,y,scoring='neg_log_loss',cv=5)
    print('mean score:',cv.mean())
    print('std:',cv.std())
    print('------------------')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=9, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

LOG LOSS: 2.004232514555692
mean score: -2.0174231312186572
std: 0.046835399000566914
------------------


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=9, splitter='best')

LOG LOSS: 1.520959687324562
mean score: -1.34044868446975
std: 0.24614062575749449
------------------


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=9, verbose=0,
                       warm_start=False)

...

212:	learn: 0.8712191	total: 5.47s	remaining: 20.2s
213:	learn: 0.8699652	total: 5.49s	remaining: 20.2s
214:	learn: 0.8692118	total: 5.52s	remaining: 20.1s
215:	learn: 0.8686729	total: 5.55s	remaining: 20.1s
216:	learn: 0.8681426	total: 5.55s	remaining: 20s
217:	learn: 0.8676176	total: 5.58s	remaining: 20s
218:	learn: 0.8668813	total: 5.59s	remaining: 20s
219:	learn: 0.8657721	total: 5.62s	remaining: 19.9s
220:	learn: 0.8646301	total: 5.63s	remaining: 19.9s
221:	learn: 0.8631247	total: 5.67s	remaining: 19.9s
222:	learn: 0.8626110	total: 5.7s	remaining: 19.9s
223:	learn: 0.8618157	total: 5.72s	remaining: 19.8s
224:	learn: 0.8603051	total: 5.74s	remaining: 19.8s
225:	learn: 0.8590015	total: 5.76s	remaining: 19.7s
226:	learn: 0.8579441	total: 5.8s	remaining: 19.8s
227:	learn: 0.8572681	total: 5.83s	remaining: 19.7s
228:	learn: 0.8568231	total: 5.86s	remaining: 19.7s
229:	learn: 0.8557287	total: 5.88s	remaining: 19.7s
230:	learn: 0.8542618	total: 5.9s	remaining: 19.7s
231:	learn: 0.8536965

376:	learn: 0.7495871	total: 9.56s	remaining: 15.8s
377:	learn: 0.7492266	total: 9.59s	remaining: 15.8s
378:	learn: 0.7484713	total: 9.62s	remaining: 15.8s
379:	learn: 0.7480739	total: 9.64s	remaining: 15.7s
380:	learn: 0.7475172	total: 9.66s	remaining: 15.7s
381:	learn: 0.7466615	total: 9.68s	remaining: 15.7s
382:	learn: 0.7464239	total: 9.7s	remaining: 15.6s
383:	learn: 0.7457095	total: 9.72s	remaining: 15.6s
384:	learn: 0.7454531	total: 9.74s	remaining: 15.6s
385:	learn: 0.7449297	total: 9.77s	remaining: 15.5s
386:	learn: 0.7439851	total: 9.79s	remaining: 15.5s
387:	learn: 0.7436875	total: 9.81s	remaining: 15.5s
388:	learn: 0.7433839	total: 9.83s	remaining: 15.4s
389:	learn: 0.7431158	total: 9.85s	remaining: 15.4s
390:	learn: 0.7426509	total: 9.88s	remaining: 15.4s
391:	learn: 0.7424656	total: 9.89s	remaining: 15.3s
392:	learn: 0.7423477	total: 9.91s	remaining: 15.3s
393:	learn: 0.7419607	total: 9.94s	remaining: 15.3s
394:	learn: 0.7417183	total: 9.96s	remaining: 15.3s
395:	learn: 0

540:	learn: 0.6913075	total: 13.3s	remaining: 11.3s
541:	learn: 0.6910323	total: 13.3s	remaining: 11.3s
542:	learn: 0.6905756	total: 13.4s	remaining: 11.3s
543:	learn: 0.6904673	total: 13.4s	remaining: 11.2s
544:	learn: 0.6902430	total: 13.4s	remaining: 11.2s
545:	learn: 0.6900668	total: 13.4s	remaining: 11.2s
546:	learn: 0.6897388	total: 13.5s	remaining: 11.1s
547:	learn: 0.6895820	total: 13.5s	remaining: 11.1s
548:	learn: 0.6891885	total: 13.5s	remaining: 11.1s
549:	learn: 0.6888371	total: 13.5s	remaining: 11.1s
550:	learn: 0.6886602	total: 13.6s	remaining: 11s
551:	learn: 0.6884546	total: 13.6s	remaining: 11s
552:	learn: 0.6883563	total: 13.6s	remaining: 11s
553:	learn: 0.6879845	total: 13.6s	remaining: 11s
554:	learn: 0.6876994	total: 13.6s	remaining: 10.9s
555:	learn: 0.6875040	total: 13.7s	remaining: 10.9s
556:	learn: 0.6872922	total: 13.7s	remaining: 10.9s
557:	learn: 0.6871713	total: 13.7s	remaining: 10.9s
558:	learn: 0.6868699	total: 13.7s	remaining: 10.8s
559:	learn: 0.686777

703:	learn: 0.6601333	total: 17.1s	remaining: 7.2s
704:	learn: 0.6599980	total: 17.1s	remaining: 7.18s
705:	learn: 0.6597682	total: 17.2s	remaining: 7.15s
706:	learn: 0.6596863	total: 17.2s	remaining: 7.13s
707:	learn: 0.6595032	total: 17.2s	remaining: 7.11s
708:	learn: 0.6593142	total: 17.3s	remaining: 7.08s
709:	learn: 0.6592202	total: 17.3s	remaining: 7.06s
710:	learn: 0.6591145	total: 17.3s	remaining: 7.04s
711:	learn: 0.6589624	total: 17.4s	remaining: 7.02s
712:	learn: 0.6588607	total: 17.4s	remaining: 7s
713:	learn: 0.6586278	total: 17.4s	remaining: 6.97s
714:	learn: 0.6585663	total: 17.4s	remaining: 6.95s
715:	learn: 0.6584498	total: 17.5s	remaining: 6.92s
716:	learn: 0.6583629	total: 17.5s	remaining: 6.9s
717:	learn: 0.6581781	total: 17.5s	remaining: 6.88s
718:	learn: 0.6580936	total: 17.5s	remaining: 6.86s
719:	learn: 0.6578439	total: 17.6s	remaining: 6.83s
720:	learn: 0.6577517	total: 17.6s	remaining: 6.81s
721:	learn: 0.6575989	total: 17.6s	remaining: 6.79s
722:	learn: 0.657

864:	learn: 0.6402006	total: 21.2s	remaining: 3.31s
865:	learn: 0.6400624	total: 21.3s	remaining: 3.29s
866:	learn: 0.6400184	total: 21.3s	remaining: 3.27s
867:	learn: 0.6399471	total: 21.3s	remaining: 3.24s
868:	learn: 0.6398758	total: 21.3s	remaining: 3.22s
869:	learn: 0.6398391	total: 21.4s	remaining: 3.19s
870:	learn: 0.6397447	total: 21.4s	remaining: 3.17s
871:	learn: 0.6395503	total: 21.4s	remaining: 3.15s
872:	learn: 0.6394445	total: 21.5s	remaining: 3.12s
873:	learn: 0.6394047	total: 21.5s	remaining: 3.1s
874:	learn: 0.6393478	total: 21.5s	remaining: 3.07s
875:	learn: 0.6392551	total: 21.5s	remaining: 3.05s
876:	learn: 0.6391599	total: 21.6s	remaining: 3.02s
877:	learn: 0.6390582	total: 21.6s	remaining: 3s
878:	learn: 0.6388750	total: 21.6s	remaining: 2.98s
879:	learn: 0.6387622	total: 21.6s	remaining: 2.95s
880:	learn: 0.6387140	total: 21.7s	remaining: 2.92s
881:	learn: 0.6386612	total: 21.7s	remaining: 2.9s
882:	learn: 0.6385573	total: 21.7s	remaining: 2.87s
883:	learn: 0.638

In [216]:
#voting classifier
from sklearn.ensemble import VotingClassifier
X2=new_train_poly.drop('MatchWinner',1)
y2=new_train_poly['MatchWinner']
#______________________________________________________________________________________________________________________
x_train,x_test,y_train,y_test= train_test_split(X2,y2,test_size=0.3,random_state=9)
#x_train_sm4, y_train_sm4 = smote.fit_sample(x_train,y_train)

#______________________________________________________________________________________________________________________
voting_clf_soft_2=VotingClassifier(estimators =[('xgb',XGBClassifier(random_state=9)),('lgb',LGBMClassifier(random_state=9)),('cat',CatBoostClassifier(random_state=9))], voting = 'soft')
voting_clf_soft_2.fit(x_train, y_train)
y_pred_vote=voting_clf_soft_2.predict_proba(x_test)
LOG_LOSS=log_loss(y_test,voting_clf_soft_2.predict_proba(x_test))
#print(model)
print('LOG LOSS:',LOG_LOSS)
cv=cross_val_score(voting_clf_soft_2,X2,y2,scoring='neg_log_loss',cv=5)
print('mean score:',cv.mean())
print('std:',cv.std())
print('------------------')



0:	learn: 2.6980519	total: 92.5ms	remaining: 1m 32s
1:	learn: 2.6226615	total: 132ms	remaining: 1m 5s
2:	learn: 2.5608589	total: 155ms	remaining: 51.5s
3:	learn: 2.5064852	total: 175ms	remaining: 43.5s
4:	learn: 2.4573416	total: 195ms	remaining: 38.8s
5:	learn: 2.4139497	total: 215ms	remaining: 35.6s
6:	learn: 2.3628715	total: 235ms	remaining: 33.3s
7:	learn: 2.3240214	total: 254ms	remaining: 31.5s
8:	learn: 2.2867382	total: 274ms	remaining: 30.2s
9:	learn: 2.2484972	total: 300ms	remaining: 29.7s
10:	learn: 2.2167331	total: 328ms	remaining: 29.5s
11:	learn: 2.1848170	total: 372ms	remaining: 30.7s
12:	learn: 2.1512560	total: 409ms	remaining: 31s
13:	learn: 2.1240108	total: 431ms	remaining: 30.3s
14:	learn: 2.0979400	total: 450ms	remaining: 29.6s
15:	learn: 2.0712225	total: 471ms	remaining: 29s
16:	learn: 2.0418035	total: 493ms	remaining: 28.5s
17:	learn: 2.0123191	total: 519ms	remaining: 28.3s
18:	learn: 1.9873781	total: 549ms	remaining: 28.4s
19:	learn: 1.9640476	total: 587ms	remaining

162:	learn: 0.9635231	total: 4.02s	remaining: 20.6s
163:	learn: 0.9613497	total: 4.05s	remaining: 20.6s
164:	learn: 0.9594273	total: 4.07s	remaining: 20.6s
165:	learn: 0.9574961	total: 4.11s	remaining: 20.6s
166:	learn: 0.9554207	total: 4.14s	remaining: 20.6s
167:	learn: 0.9535412	total: 4.16s	remaining: 20.6s
168:	learn: 0.9515728	total: 4.18s	remaining: 20.6s
169:	learn: 0.9493087	total: 4.21s	remaining: 20.5s
170:	learn: 0.9475722	total: 4.23s	remaining: 20.5s
171:	learn: 0.9461465	total: 4.26s	remaining: 20.5s
172:	learn: 0.9448525	total: 4.29s	remaining: 20.5s
173:	learn: 0.9439702	total: 4.31s	remaining: 20.5s
174:	learn: 0.9427713	total: 4.33s	remaining: 20.4s
175:	learn: 0.9415633	total: 4.35s	remaining: 20.4s
176:	learn: 0.9399534	total: 4.37s	remaining: 20.3s
177:	learn: 0.9381063	total: 4.4s	remaining: 20.3s
178:	learn: 0.9373468	total: 4.42s	remaining: 20.3s
179:	learn: 0.9365846	total: 4.45s	remaining: 20.3s
180:	learn: 0.9353383	total: 4.48s	remaining: 20.3s
181:	learn: 0

321:	learn: 0.7908983	total: 7.79s	remaining: 16.4s
322:	learn: 0.7901852	total: 7.81s	remaining: 16.4s
323:	learn: 0.7895887	total: 7.84s	remaining: 16.4s
324:	learn: 0.7887846	total: 7.86s	remaining: 16.3s
325:	learn: 0.7884483	total: 7.89s	remaining: 16.3s
326:	learn: 0.7880997	total: 7.91s	remaining: 16.3s
327:	learn: 0.7877160	total: 7.93s	remaining: 16.2s
328:	learn: 0.7870268	total: 7.95s	remaining: 16.2s
329:	learn: 0.7861698	total: 7.97s	remaining: 16.2s
330:	learn: 0.7859313	total: 8s	remaining: 16.2s
331:	learn: 0.7853228	total: 8.02s	remaining: 16.1s
332:	learn: 0.7850834	total: 8.04s	remaining: 16.1s
333:	learn: 0.7842672	total: 8.06s	remaining: 16.1s
334:	learn: 0.7834582	total: 8.08s	remaining: 16s
335:	learn: 0.7826008	total: 8.1s	remaining: 16s
336:	learn: 0.7823629	total: 8.12s	remaining: 16s
337:	learn: 0.7814587	total: 8.14s	remaining: 15.9s
338:	learn: 0.7807348	total: 8.17s	remaining: 15.9s
339:	learn: 0.7803877	total: 8.19s	remaining: 15.9s
340:	learn: 0.7799396	

482:	learn: 0.7192041	total: 11.4s	remaining: 12.2s
483:	learn: 0.7188534	total: 11.4s	remaining: 12.2s
484:	learn: 0.7185497	total: 11.4s	remaining: 12.1s
485:	learn: 0.7181999	total: 11.5s	remaining: 12.1s
486:	learn: 0.7180263	total: 11.5s	remaining: 12.1s
487:	learn: 0.7175208	total: 11.5s	remaining: 12.1s
488:	learn: 0.7173629	total: 11.5s	remaining: 12s
489:	learn: 0.7172002	total: 11.5s	remaining: 12s
490:	learn: 0.7170346	total: 11.6s	remaining: 12s
491:	learn: 0.7168018	total: 11.6s	remaining: 12s
492:	learn: 0.7166456	total: 11.6s	remaining: 11.9s
493:	learn: 0.7162718	total: 11.6s	remaining: 11.9s
494:	learn: 0.7157745	total: 11.7s	remaining: 11.9s
495:	learn: 0.7154350	total: 11.7s	remaining: 11.9s
496:	learn: 0.7152861	total: 11.7s	remaining: 11.8s
497:	learn: 0.7151678	total: 11.7s	remaining: 11.8s
498:	learn: 0.7149506	total: 11.7s	remaining: 11.8s
499:	learn: 0.7146920	total: 11.8s	remaining: 11.8s
500:	learn: 0.7143615	total: 11.8s	remaining: 11.7s
501:	learn: 0.714104

642:	learn: 0.6801438	total: 14.9s	remaining: 8.29s
643:	learn: 0.6798198	total: 15s	remaining: 8.28s
644:	learn: 0.6796581	total: 15s	remaining: 8.26s
645:	learn: 0.6794550	total: 15s	remaining: 8.23s
646:	learn: 0.6793582	total: 15s	remaining: 8.21s
647:	learn: 0.6792290	total: 15.1s	remaining: 8.18s
648:	learn: 0.6791295	total: 15.1s	remaining: 8.16s
649:	learn: 0.6789113	total: 15.1s	remaining: 8.13s
650:	learn: 0.6787339	total: 15.1s	remaining: 8.11s
651:	learn: 0.6786591	total: 15.1s	remaining: 8.08s
652:	learn: 0.6784662	total: 15.2s	remaining: 8.06s
653:	learn: 0.6782612	total: 15.2s	remaining: 8.04s
654:	learn: 0.6780493	total: 15.2s	remaining: 8.02s
655:	learn: 0.6778242	total: 15.2s	remaining: 7.99s
656:	learn: 0.6776500	total: 15.3s	remaining: 7.97s
657:	learn: 0.6774302	total: 15.3s	remaining: 7.94s
658:	learn: 0.6771325	total: 15.3s	remaining: 7.92s
659:	learn: 0.6769157	total: 15.3s	remaining: 7.9s
660:	learn: 0.6768414	total: 15.4s	remaining: 7.87s
661:	learn: 0.6767469

...

206:	learn: 0.8787896	total: 4.89s	remaining: 18.7s
207:	learn: 0.8777631	total: 4.92s	remaining: 18.8s
208:	learn: 0.8759579	total: 4.96s	remaining: 18.8s
209:	learn: 0.8744410	total: 4.98s	remaining: 18.7s
210:	learn: 0.8729140	total: 5s	remaining: 18.7s
211:	learn: 0.8721608	total: 5.02s	remaining: 18.6s
212:	learn: 0.8712191	total: 5.04s	remaining: 18.6s
213:	learn: 0.8699652	total: 5.06s	remaining: 18.6s
214:	learn: 0.8692118	total: 5.09s	remaining: 18.6s
215:	learn: 0.8686729	total: 5.12s	remaining: 18.6s
216:	learn: 0.8681426	total: 5.12s	remaining: 18.5s
217:	learn: 0.8676176	total: 5.14s	remaining: 18.5s
218:	learn: 0.8668813	total: 5.17s	remaining: 18.4s
219:	learn: 0.8657721	total: 5.18s	remaining: 18.4s
220:	learn: 0.8646301	total: 5.21s	remaining: 18.4s
221:	learn: 0.8631247	total: 5.23s	remaining: 18.3s
222:	learn: 0.8626110	total: 5.25s	remaining: 18.3s
223:	learn: 0.8618157	total: 5.28s	remaining: 18.3s
224:	learn: 0.8603051	total: 5.3s	remaining: 18.3s
225:	learn: 0.85

371:	learn: 0.7518152	total: 8.62s	remaining: 14.6s
372:	learn: 0.7514687	total: 8.65s	remaining: 14.5s
373:	learn: 0.7512907	total: 8.68s	remaining: 14.5s
374:	learn: 0.7508532	total: 8.71s	remaining: 14.5s
375:	learn: 0.7501878	total: 8.72s	remaining: 14.5s
376:	learn: 0.7495871	total: 8.74s	remaining: 14.5s
377:	learn: 0.7492266	total: 8.77s	remaining: 14.4s
378:	learn: 0.7484713	total: 8.79s	remaining: 14.4s
379:	learn: 0.7480739	total: 8.81s	remaining: 14.4s
380:	learn: 0.7475172	total: 8.84s	remaining: 14.4s
381:	learn: 0.7466615	total: 8.87s	remaining: 14.3s
382:	learn: 0.7464239	total: 8.88s	remaining: 14.3s
383:	learn: 0.7457095	total: 8.9s	remaining: 14.3s
384:	learn: 0.7454531	total: 8.93s	remaining: 14.3s
385:	learn: 0.7449297	total: 8.95s	remaining: 14.2s
386:	learn: 0.7439851	total: 8.97s	remaining: 14.2s
387:	learn: 0.7436875	total: 8.99s	remaining: 14.2s
388:	learn: 0.7433839	total: 9.02s	remaining: 14.2s
389:	learn: 0.7431158	total: 9.04s	remaining: 14.1s
390:	learn: 0

532:	learn: 0.6934800	total: 12.3s	remaining: 10.8s
533:	learn: 0.6932551	total: 12.3s	remaining: 10.8s
534:	learn: 0.6927565	total: 12.4s	remaining: 10.8s
535:	learn: 0.6926353	total: 12.4s	remaining: 10.7s
536:	learn: 0.6925215	total: 12.4s	remaining: 10.7s
537:	learn: 0.6922607	total: 12.4s	remaining: 10.7s
538:	learn: 0.6920021	total: 12.5s	remaining: 10.7s
539:	learn: 0.6917579	total: 12.5s	remaining: 10.6s
540:	learn: 0.6913075	total: 12.5s	remaining: 10.6s
541:	learn: 0.6910323	total: 12.5s	remaining: 10.6s
542:	learn: 0.6905756	total: 12.6s	remaining: 10.6s
543:	learn: 0.6904673	total: 12.6s	remaining: 10.5s
544:	learn: 0.6902430	total: 12.6s	remaining: 10.5s
545:	learn: 0.6900668	total: 12.6s	remaining: 10.5s
546:	learn: 0.6897388	total: 12.6s	remaining: 10.5s
547:	learn: 0.6895820	total: 12.7s	remaining: 10.4s
548:	learn: 0.6891885	total: 12.7s	remaining: 10.4s
549:	learn: 0.6888371	total: 12.7s	remaining: 10.4s
550:	learn: 0.6886602	total: 12.7s	remaining: 10.4s
551:	learn: 

693:	learn: 0.6616342	total: 16.2s	remaining: 7.15s
694:	learn: 0.6614966	total: 16.3s	remaining: 7.14s
695:	learn: 0.6612733	total: 16.3s	remaining: 7.12s
696:	learn: 0.6610186	total: 16.3s	remaining: 7.1s
697:	learn: 0.6609170	total: 16.3s	remaining: 7.07s
698:	learn: 0.6608003	total: 16.4s	remaining: 7.05s
699:	learn: 0.6606114	total: 16.4s	remaining: 7.03s
700:	learn: 0.6604737	total: 16.4s	remaining: 7.01s
701:	learn: 0.6602838	total: 16.5s	remaining: 6.99s
702:	learn: 0.6601694	total: 16.5s	remaining: 6.96s
703:	learn: 0.6601333	total: 16.5s	remaining: 6.93s
704:	learn: 0.6599980	total: 16.5s	remaining: 6.91s
705:	learn: 0.6597682	total: 16.5s	remaining: 6.89s
706:	learn: 0.6596863	total: 16.6s	remaining: 6.86s
707:	learn: 0.6595032	total: 16.6s	remaining: 6.84s
708:	learn: 0.6593142	total: 16.6s	remaining: 6.82s
709:	learn: 0.6592202	total: 16.6s	remaining: 6.8s
710:	learn: 0.6591145	total: 16.7s	remaining: 6.78s
711:	learn: 0.6589624	total: 16.7s	remaining: 6.75s
712:	learn: 0.

852:	learn: 0.6413606	total: 20.1s	remaining: 3.46s
853:	learn: 0.6411945	total: 20.1s	remaining: 3.44s
854:	learn: 0.6410404	total: 20.2s	remaining: 3.42s
855:	learn: 0.6409177	total: 20.2s	remaining: 3.39s
856:	learn: 0.6408301	total: 20.2s	remaining: 3.37s
857:	learn: 0.6407565	total: 20.2s	remaining: 3.35s
858:	learn: 0.6406908	total: 20.2s	remaining: 3.32s
859:	learn: 0.6406256	total: 20.3s	remaining: 3.3s
860:	learn: 0.6405521	total: 20.3s	remaining: 3.28s
861:	learn: 0.6404930	total: 20.3s	remaining: 3.25s
862:	learn: 0.6404470	total: 20.4s	remaining: 3.23s
863:	learn: 0.6404011	total: 20.4s	remaining: 3.21s
864:	learn: 0.6402006	total: 20.4s	remaining: 3.18s
865:	learn: 0.6400624	total: 20.4s	remaining: 3.16s
866:	learn: 0.6400184	total: 20.5s	remaining: 3.14s
867:	learn: 0.6399471	total: 20.5s	remaining: 3.11s
868:	learn: 0.6398758	total: 20.5s	remaining: 3.09s
869:	learn: 0.6398391	total: 20.5s	remaining: 3.07s
870:	learn: 0.6397447	total: 20.6s	remaining: 3.04s
871:	learn: 0

In [203]:
from sklearn.ensemble import StackingClassifier

X=new_train_poly.drop('MatchWinner',1)
y=new_train_poly['MatchWinner']

# define the base models
level0 = list()
level0.append(('xgb', XGBClassifier(random_state=9)
))
level0.append(('lgb',LGBMClassifier(random_state=9)))
level0.append(('cat',CatBoostClassifier(random_state=9)))
# define meta learner model
level1 =XGBClassifier(random_state=9)
X=new_train.drop('MatchWinner',1)
y=(new_train['MatchWinner'])
#X.fillna(0,inplace=True)
# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=9)

# fit the model on all available data
model.fit(X_train,y_train)

y_pred=model.predict_proba(X_test)

LOG_LOSS=log_loss(y_test,model.predict_proba(X_test))
#print(model)
print('LOG LOSS:',LOG_LOSS)
cv=cross_val_score(model,X,y,scoring='neg_log_loss',cv=5)
print('mean score:',cv.mean())
print('std:',cv.std())
print('------------------')

0:	learn: 2.6981491	total: 38.1ms	remaining: 38.1s
1:	learn: 2.6277782	total: 43.8ms	remaining: 21.9s
2:	learn: 2.5615466	total: 49.4ms	remaining: 16.4s
3:	learn: 2.5060840	total: 55ms	remaining: 13.7s
4:	learn: 2.4549681	total: 64.4ms	remaining: 12.8s
5:	learn: 2.4032803	total: 70.3ms	remaining: 11.7s
6:	learn: 2.3541631	total: 75.8ms	remaining: 10.7s
7:	learn: 2.3138320	total: 81.1ms	remaining: 10.1s
8:	learn: 2.2707473	total: 86.5ms	remaining: 9.53s
9:	learn: 2.2346490	total: 91.8ms	remaining: 9.09s
10:	learn: 2.1968589	total: 96.7ms	remaining: 8.69s
11:	learn: 2.1621615	total: 102ms	remaining: 8.38s
12:	learn: 2.1293476	total: 107ms	remaining: 8.12s
13:	learn: 2.0950285	total: 112ms	remaining: 7.88s
14:	learn: 2.0630736	total: 117ms	remaining: 7.69s
15:	learn: 2.0350207	total: 123ms	remaining: 7.54s
16:	learn: 2.0070099	total: 127ms	remaining: 7.36s
17:	learn: 1.9793108	total: 132ms	remaining: 7.22s
18:	learn: 1.9544275	total: 137ms	remaining: 7.07s
19:	learn: 1.9302112	total: 142m

165:	learn: 0.9289437	total: 1.07s	remaining: 5.39s
166:	learn: 0.9269703	total: 1.08s	remaining: 5.38s
167:	learn: 0.9246537	total: 1.08s	remaining: 5.37s
168:	learn: 0.9232182	total: 1.09s	remaining: 5.36s
169:	learn: 0.9215531	total: 1.1s	remaining: 5.36s
170:	learn: 0.9200252	total: 1.1s	remaining: 5.35s
171:	learn: 0.9188236	total: 1.11s	remaining: 5.34s
172:	learn: 0.9174826	total: 1.12s	remaining: 5.34s
173:	learn: 0.9160251	total: 1.12s	remaining: 5.34s
174:	learn: 0.9139923	total: 1.14s	remaining: 5.36s
175:	learn: 0.9129858	total: 1.14s	remaining: 5.36s
176:	learn: 0.9116647	total: 1.15s	remaining: 5.36s
177:	learn: 0.9097601	total: 1.16s	remaining: 5.35s
178:	learn: 0.9082140	total: 1.16s	remaining: 5.34s
179:	learn: 0.9069375	total: 1.17s	remaining: 5.33s
180:	learn: 0.9049380	total: 1.18s	remaining: 5.32s
181:	learn: 0.9028164	total: 1.18s	remaining: 5.31s
182:	learn: 0.9015628	total: 1.19s	remaining: 5.3s
183:	learn: 0.8998509	total: 1.19s	remaining: 5.28s
184:	learn: 0.8

347:	learn: 0.7499758	total: 2.08s	remaining: 3.91s
348:	learn: 0.7491054	total: 2.09s	remaining: 3.9s
349:	learn: 0.7484354	total: 2.1s	remaining: 3.89s
350:	learn: 0.7479355	total: 2.1s	remaining: 3.89s
351:	learn: 0.7473854	total: 2.11s	remaining: 3.88s
352:	learn: 0.7471037	total: 2.11s	remaining: 3.87s
353:	learn: 0.7466603	total: 2.12s	remaining: 3.87s
354:	learn: 0.7463611	total: 2.12s	remaining: 3.86s
355:	learn: 0.7456600	total: 2.13s	remaining: 3.85s
356:	learn: 0.7449434	total: 2.13s	remaining: 3.85s
357:	learn: 0.7442124	total: 2.14s	remaining: 3.84s
358:	learn: 0.7435085	total: 2.15s	remaining: 3.83s
359:	learn: 0.7430404	total: 2.16s	remaining: 3.83s
360:	learn: 0.7427175	total: 2.16s	remaining: 3.83s
361:	learn: 0.7420164	total: 2.17s	remaining: 3.82s
362:	learn: 0.7414760	total: 2.17s	remaining: 3.81s
363:	learn: 0.7409651	total: 2.18s	remaining: 3.81s
364:	learn: 0.7405921	total: 2.18s	remaining: 3.8s
365:	learn: 0.7399148	total: 2.19s	remaining: 3.79s
366:	learn: 0.73

526:	learn: 0.6802802	total: 3.1s	remaining: 2.79s
527:	learn: 0.6799574	total: 3.11s	remaining: 2.78s
528:	learn: 0.6796558	total: 3.12s	remaining: 2.77s
529:	learn: 0.6795175	total: 3.12s	remaining: 2.77s
530:	learn: 0.6793086	total: 3.13s	remaining: 2.76s
531:	learn: 0.6791667	total: 3.13s	remaining: 2.75s
532:	learn: 0.6788517	total: 3.14s	remaining: 2.75s
533:	learn: 0.6786895	total: 3.14s	remaining: 2.74s
534:	learn: 0.6784548	total: 3.15s	remaining: 2.74s
535:	learn: 0.6781241	total: 3.15s	remaining: 2.73s
536:	learn: 0.6779311	total: 3.16s	remaining: 2.72s
537:	learn: 0.6775638	total: 3.17s	remaining: 2.72s
538:	learn: 0.6772653	total: 3.18s	remaining: 2.72s
539:	learn: 0.6769604	total: 3.18s	remaining: 2.71s
540:	learn: 0.6767823	total: 3.19s	remaining: 2.71s
541:	learn: 0.6766020	total: 3.19s	remaining: 2.7s
542:	learn: 0.6763052	total: 3.2s	remaining: 2.69s
543:	learn: 0.6760559	total: 3.21s	remaining: 2.69s
544:	learn: 0.6758339	total: 3.21s	remaining: 2.68s
545:	learn: 0.6

705:	learn: 0.6450922	total: 4.09s	remaining: 1.7s
706:	learn: 0.6450035	total: 4.1s	remaining: 1.7s
707:	learn: 0.6448186	total: 4.11s	remaining: 1.69s
708:	learn: 0.6446577	total: 4.11s	remaining: 1.69s
709:	learn: 0.6445196	total: 4.12s	remaining: 1.68s
710:	learn: 0.6443988	total: 4.12s	remaining: 1.67s
711:	learn: 0.6442905	total: 4.13s	remaining: 1.67s
712:	learn: 0.6441874	total: 4.13s	remaining: 1.66s
713:	learn: 0.6440867	total: 4.14s	remaining: 1.66s
714:	learn: 0.6439735	total: 4.14s	remaining: 1.65s
715:	learn: 0.6438489	total: 4.15s	remaining: 1.65s
716:	learn: 0.6436855	total: 4.15s	remaining: 1.64s
717:	learn: 0.6435692	total: 4.16s	remaining: 1.63s
718:	learn: 0.6433748	total: 4.17s	remaining: 1.63s
719:	learn: 0.6432063	total: 4.17s	remaining: 1.62s
720:	learn: 0.6430781	total: 4.18s	remaining: 1.62s
721:	learn: 0.6429131	total: 4.18s	remaining: 1.61s
722:	learn: 0.6428046	total: 4.19s	remaining: 1.6s
723:	learn: 0.6427271	total: 4.19s	remaining: 1.6s
724:	learn: 0.642

...

282:	learn: 0.7793115	total: 1.86s	remaining: 4.72s
283:	learn: 0.7787126	total: 1.87s	remaining: 4.71s
284:	learn: 0.7782622	total: 1.87s	remaining: 4.7s
285:	learn: 0.7776464	total: 1.89s	remaining: 4.71s
286:	learn: 0.7768913	total: 1.89s	remaining: 4.7s
287:	learn: 0.7760709	total: 1.9s	remaining: 4.69s
288:	learn: 0.7754606	total: 1.9s	remaining: 4.68s
289:	learn: 0.7743226	total: 1.91s	remaining: 4.67s
290:	learn: 0.7735411	total: 1.92s	remaining: 4.67s
291:	learn: 0.7728213	total: 1.92s	remaining: 4.66s
292:	learn: 0.7721829	total: 1.93s	remaining: 4.66s
293:	learn: 0.7717573	total: 1.94s	remaining: 4.65s
294:	learn: 0.7708090	total: 1.94s	remaining: 4.64s
295:	learn: 0.7699168	total: 1.95s	remaining: 4.64s
296:	learn: 0.7692668	total: 1.96s	remaining: 4.63s
297:	learn: 0.7687756	total: 1.96s	remaining: 4.62s
298:	learn: 0.7681967	total: 1.97s	remaining: 4.62s
299:	learn: 0.7677015	total: 1.97s	remaining: 4.61s
300:	learn: 0.7669072	total: 1.99s	remaining: 4.61s
301:	learn: 0.76

442:	learn: 0.6917981	total: 2.87s	remaining: 3.61s
443:	learn: 0.6915725	total: 2.87s	remaining: 3.6s
444:	learn: 0.6910402	total: 2.88s	remaining: 3.59s
445:	learn: 0.6908721	total: 2.88s	remaining: 3.58s
446:	learn: 0.6903442	total: 2.89s	remaining: 3.57s
447:	learn: 0.6900422	total: 2.89s	remaining: 3.57s
448:	learn: 0.6898297	total: 2.9s	remaining: 3.56s
449:	learn: 0.6893749	total: 2.91s	remaining: 3.56s
450:	learn: 0.6888446	total: 2.92s	remaining: 3.55s
451:	learn: 0.6885534	total: 2.92s	remaining: 3.54s
452:	learn: 0.6883710	total: 2.93s	remaining: 3.53s
453:	learn: 0.6880399	total: 2.93s	remaining: 3.53s
454:	learn: 0.6877868	total: 2.94s	remaining: 3.52s
455:	learn: 0.6874708	total: 2.95s	remaining: 3.51s
456:	learn: 0.6869146	total: 2.95s	remaining: 3.51s
457:	learn: 0.6865560	total: 2.96s	remaining: 3.5s
458:	learn: 0.6863603	total: 2.96s	remaining: 3.49s
459:	learn: 0.6859413	total: 2.96s	remaining: 3.48s
460:	learn: 0.6855019	total: 2.97s	remaining: 3.47s
461:	learn: 0.6

630:	learn: 0.6399810	total: 3.88s	remaining: 2.27s
631:	learn: 0.6398590	total: 3.89s	remaining: 2.26s
632:	learn: 0.6397449	total: 3.89s	remaining: 2.26s
633:	learn: 0.6395669	total: 3.9s	remaining: 2.25s
634:	learn: 0.6393871	total: 3.9s	remaining: 2.24s
635:	learn: 0.6392636	total: 3.91s	remaining: 2.24s
636:	learn: 0.6390252	total: 3.91s	remaining: 2.23s
637:	learn: 0.6387888	total: 3.92s	remaining: 2.22s
638:	learn: 0.6385152	total: 3.92s	remaining: 2.22s
639:	learn: 0.6383816	total: 3.93s	remaining: 2.21s
640:	learn: 0.6381983	total: 3.93s	remaining: 2.2s
641:	learn: 0.6380085	total: 3.94s	remaining: 2.2s
642:	learn: 0.6378757	total: 3.95s	remaining: 2.19s
643:	learn: 0.6376597	total: 3.95s	remaining: 2.18s
644:	learn: 0.6375569	total: 3.96s	remaining: 2.18s
645:	learn: 0.6373622	total: 3.96s	remaining: 2.17s
646:	learn: 0.6372021	total: 3.97s	remaining: 2.17s
647:	learn: 0.6370793	total: 3.97s	remaining: 2.16s
648:	learn: 0.6369578	total: 3.98s	remaining: 2.15s
649:	learn: 0.63

810:	learn: 0.6140926	total: 4.89s	remaining: 1.14s
811:	learn: 0.6139955	total: 4.89s	remaining: 1.13s
812:	learn: 0.6138447	total: 4.9s	remaining: 1.13s
813:	learn: 0.6137390	total: 4.9s	remaining: 1.12s
814:	learn: 0.6135853	total: 4.91s	remaining: 1.11s
815:	learn: 0.6134860	total: 4.91s	remaining: 1.11s
816:	learn: 0.6133839	total: 4.92s	remaining: 1.1s
817:	learn: 0.6132817	total: 4.92s	remaining: 1.09s
818:	learn: 0.6132069	total: 4.93s	remaining: 1.09s
819:	learn: 0.6130908	total: 4.94s	remaining: 1.08s
820:	learn: 0.6130119	total: 4.94s	remaining: 1.08s
821:	learn: 0.6128971	total: 4.95s	remaining: 1.07s
822:	learn: 0.6128082	total: 4.95s	remaining: 1.06s
823:	learn: 0.6127205	total: 4.96s	remaining: 1.06s
824:	learn: 0.6126078	total: 4.96s	remaining: 1.05s
825:	learn: 0.6124880	total: 4.97s	remaining: 1.05s
826:	learn: 0.6124065	total: 4.97s	remaining: 1.04s
827:	learn: 0.6122726	total: 4.98s	remaining: 1.03s
828:	learn: 0.6121787	total: 4.99s	remaining: 1.03s
829:	learn: 0.6

969:	learn: 0.5995844	total: 5.73s	remaining: 177ms
970:	learn: 0.5994988	total: 5.74s	remaining: 171ms
971:	learn: 0.5993938	total: 5.74s	remaining: 165ms
972:	learn: 0.5993034	total: 5.75s	remaining: 159ms
973:	learn: 0.5992155	total: 5.75s	remaining: 154ms
974:	learn: 0.5991080	total: 5.76s	remaining: 148ms
975:	learn: 0.5990355	total: 5.76s	remaining: 142ms
976:	learn: 0.5989449	total: 5.77s	remaining: 136ms
977:	learn: 0.5988765	total: 5.78s	remaining: 130ms
978:	learn: 0.5988167	total: 5.78s	remaining: 124ms
979:	learn: 0.5987546	total: 5.79s	remaining: 118ms
980:	learn: 0.5986765	total: 5.79s	remaining: 112ms
981:	learn: 0.5986084	total: 5.8s	remaining: 106ms
982:	learn: 0.5985600	total: 5.81s	remaining: 100ms
983:	learn: 0.5984669	total: 5.81s	remaining: 94.5ms
984:	learn: 0.5983985	total: 5.82s	remaining: 88.6ms
985:	learn: 0.5983154	total: 5.82s	remaining: 82.7ms
986:	learn: 0.5982674	total: 5.83s	remaining: 76.8ms
987:	learn: 0.5982178	total: 5.83s	remaining: 70.8ms
988:	lea

In [217]:
new_poly_transform_test=poly.transform(new_test)
new_poly_transform_test=pd.DataFrame(new_poly_transform_test)
new_poly_transform_test

,0,1,2,3,4,5,6,7,8,9
0,1.00,138.00,193.00,19044.00,26634.00,37249.00,2628072.00,3675492.00,5140362.00,7189057.00
1,1.00,171.00,204.00,29241.00,34884.00,41616.00,5000211.00,5965164.00,7116336.00,8489664.00
2,1.00,253.00,316.00,64009.00,79948.00,99856.00,16194277.00,20226844.00,25263568.00,31554496.00
3,1.00,253.00,316.00,64009.00,79948.00,99856.00,16194277.00,20226844.00,25263568.00,31554496.00
4,1.00,364.00,195.00,132496.00,70980.00,38025.00,48228544.00,25836720.00,13841100.00,7414875.00
...,...,...,...,...,...,...,...,...,...,...
1070,1.00,127.00,238.00,16129.00,30226.00,56644.00,2048383.00,3838702.00,7193788.00,13481272.00
1071,1.00,386.00,199.00,148996.00,76814.00,39601.00,57512456.00,29650204.00,15285986.00,7880599.00
1072,1.00,364.00,316.00,132496.00,115024.00,99856.00,48228544.00,41868736.00,36347584.00,31554496.00
1073,1.00,250.00,341.00,62500.00,85250.00,116281.00,15625000.00,21312500.00,29070250.00,39651821.00


In [219]:
y_pred=voting_clf_soft_2.predict_proba(new_poly_transform_test)

In [220]:
final=y_pred
final=pd.DataFrame(final)
final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.00,0.00,0.33,0.00,0.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.02
1,0.00,0.57,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.40,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.52,0.46,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.52,0.46,0.00,0.00,0.00,0.00,0.00
4,0.00,0.01,0.00,0.00,0.02,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,0.00,0.00,0.01,0.00,0.00,0.83,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.12
1071,0.00,0.56,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.42,0.00,0.00,0.00
1072,0.00,0.00,0.00,0.00,0.00,0.40,0.00,0.00,0.00,0.00,0.59,0.00,0.00,0.00,0.00,0.00
1073,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.63,0.00,0.00,0.35,0.00,0.00


In [221]:
final.to_excel('poly_voting.xlsx')